# Alternative chain modification approaches

Kernel: `bw25`

In notebook 2, we made the following graph, though we didn't store any activities in the database, just in the datapackage, with the "magic" id 1.000.000:

<img src="notebook-2.png">

I think this can be improved, or at least changed, in two ways. First, we can separate out the mass-balanced new aluminium mix to be more explicit, and secondly, we can store the data in the database using the `IOTable` backend.

<img src="alternative.png">

I leave everything the same (but with deleted comments) up to writing the new data.

In [4]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [6]:
bd.projects.set_current('ei38-teaching-25')

In [7]:
motor = bd.get_activity(
    database="ei 3.8 cutoff", 
    name="electric motor production, vehicle (electric powertrain)"
)
motor

'electric motor production, vehicle (electric powertrain)' (kilogram, GLO, None)

In [8]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [9]:
exio = bd.Database("EXIOBASE 3.8.1 2017 monetary")

In [10]:
alu = bd.get_activity(database="EXIOBASE 3.8.1 2017 monetary", name='Aluminium production', location='KR')

In [11]:
mapping = {
    "US": "RoW",
    "JP": "IAI Area, Asia, without China and GCC",
    "CN": "CN",
    "CA": "CA",
    "KR": "IAI Area, Asia, without China and GCC",
    "IN": "IAI Area, Asia, without China and GCC",
    "RU": "IAI Area, Russia & RER w/o EU27 & EFTA",
    "AU": "UN-OCEANIA",
    "WA": "IAI Area, Asia, without China and GCC",
    "WM": "IAI Area, Gulf Cooperation Council",
}

In [12]:
from collections import defaultdict

market = defaultdict(float)

for exc in filter(
        lambda exc: (exc.input['name'] == exc.output['name']) & (exc['amount'] > 0.005), 
        alu.technosphere()):
    market[bd.get_activity(
        database="ei 3.8 cutoff",
        name="aluminium production, primary, ingot",
        location=mapping[exc.input['location']]
    )] += exc['amount']

In [13]:
total = sum(market.values())

market = {key: value / total for key, value in market.items()}
market

{'aluminium production, primary, ingot' (kilogram, RoW, None): 0.09830303704098431,
 'aluminium production, primary, ingot' (kilogram, IAI Area, Asia, without China and GCC, None): 0.5318724848084075,
 'aluminium production, primary, ingot' (kilogram, CN, None): 0.12994627606388362,
 'aluminium production, primary, ingot' (kilogram, CA, None): 0.03721291048095496,
 'aluminium production, primary, ingot' (kilogram, IAI Area, Russia & RER w/o EU27 & EFTA, None): 0.05416918993148255,
 'aluminium production, primary, ingot' (kilogram, UN-OCEANIA, None): 0.08051082401382031,
 'aluminium production, primary, ingot' (kilogram, IAI Area, Gulf Cooperation Council, None): 0.06798527766046675}

# IOTable to the rescue

Let's create a new database to store our region-specific data:

In [16]:
kr = bd.Database("Korean Motors Chaebol", backend = "iotable")
type(kr)

bw2data.backends.iotable.backend.IOTableBackend

To make this database known to Brightway, it needs to get registered. The easiest way to is to write an empty database:

In [21]:
kr.write({})

And our activities:

In [22]:
new_motor = kr.new_activity(code="em-kr", name="Electric motor", location="KR", unit="kilogram")
new_alu = kr.new_activity(code="alu-kr", name="Korea-specific aluminium mix", location="KR", unit="kilogram")

`IOTable` backends behave differently - we don't call `write` on them, but instead first save the `Activity` objects manually, and then write the exchanges to a datapackage. First save the activities:

In [23]:
new_motor.save()
new_alu.save()

We are actually almost done. Let's assemble the edges we need. We will call the method `kr.write_exchanges()`, which has the docstring:

     Write IO data directly to processed arrays.

    Product data is stored in SQLite as normal activities.
    Exchange data is written directly to NumPy structured arrays.

    Technosphere and biosphere data are dicts with keys ``row``, ``col``, ``amount``, and ``flip``.
    
We don't have any biosphere edges, but we do have technosphere ones.

In [29]:
edges = [
    # Motor activity
    {"row": new_motor.id, "col": new_motor.id, "amount": 1},  # production exchange,
    {"row": motor.id, "col": new_motor.id, "amount": 1, "flip": True},   # Need the rest of the motor
    {"row": new_alu.id, "col": new_motor.id, "amount": 0.1685, "flip": True},  # Substitute this much aluminium
] + [
    # New alu mix
    {"row": new_alu.id, "col": new_alu.id, "amount": 1},  # production exchange,
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='aluminium ingot, primary, to aluminium, wrought alloy market'
        ).id, 
        "col": new_alu.id, 
        "amount": 1
    },   # This is the substituted aluminium mix
] + [
    {"row": node.id, "col": new_alu.id, "amount": amount, "flip": True} for node, amount in market.items()
]

In [30]:
kr.write_exchanges(edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


We do one more trick to make sure that the `Korean Motors Chaebol` database is always included when we do calculations with ecoinvent: we create a reverse edge from ecoinvent to our new database. This can have a value of zero, it just needs to be there for `Korean Motors Chaebol` to be considered a dependency of ecoinvent.

In [32]:
motor.new_edge(input=new_motor, amount=0, type="technosphere").save()

In [33]:
_, data_objs, _ = bd.prepare_lca_inputs({motor: 1}, ipcc)

In [34]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

9.134067244425333

In [35]:
lca.lcia({new_motor.id: 1})
lca.score

9.888587631052365

This seems to be working, but let's check to make sure our dependencies are set correctly:

In [36]:
bd.Database("ei 3.8 cutoff").metadata['depends']

['Korean Motors Chaebol', 'biosphere3']

In [38]:
bd.Database('Korean Motors Chaebol').metadata['depends']

['ei 3.8 cutoff']

# One more thing

This is a bit hacky, but you can abuse the `IOTable` databases to force the inclusion of datapackage data. It won't work for complicated datapackage functionality, but if it is a simple datapackage, the `write_exchanges` function **does not check** that the edge data is specific to the `IOTable` database. So, you could use the same procedure to make changes to base ecoinvent, and turn them on or off by optionally including a dummy activity from your `IOTable` backend (with an amount of zero) in your LCA calculations.